<a href="https://colab.research.google.com/github/pratikbhongade/AchPub/blob/main/5th%20June.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ask the user for the value to search and replace
search_value = input("Enter the value to search: ")

# define index for Receiving DFI Identification field in Detail Record
rdi_index = 3
batch_total_rdi_index = 10
file_total_rdi_index = 21

# read the contents of the ACH file
with open('Nacha.ach', 'r') as f:
    lines = f.readlines()

# initialize batch control total receiving DFI identification sum dictionary
batch_total_rdi = {}

# initialize file control total receiving DFI identification sum
file_total_rdi = 0
first_file_control = True

# loop through the lines of the ACH file and update the Receiving DFI Identification if found in the Detail Record
for i, line in enumerate(lines):
    record_type = line[0]
    if record_type == '5':
        # extract the batch number from the Batch Header Record
        current_batch_number = line[87:94].strip()
        # initialize the batch total receiving DFI identification sum for the current batch
        batch_total_rdi[current_batch_number] = 0

    elif record_type == '6':
        # check if the search value is present in the Receiving DFI Identification field
        rdi = line[rdi_index:rdi_index + 8]

        if search_value in rdi:
            # print the line where the search value is found
            print("Found search value in line: {}".format(line.strip()))

            # print the batch information
            entry_detail_sequence_number = line[87:94].strip()
            print("Search value is present in Batch {} and Sequence No {}.".format(current_batch_number, entry_detail_sequence_number))

            # find the positions of all occurrences of the search value within the Receiving DFI Identification field
            occurrence_positions = [index for index in range(len(rdi)) if rdi[index:index + len(search_value)] == search_value]

            # iterate through the occurrence positions and ask the user for the value to replace with
            for occurrence_index, start_index in enumerate(occurrence_positions):
                print("Occurrence {}: Position {}".format(occurrence_index + 1, start_index))
                replace_value = input("Enter the value to replace (leave blank to skip): ")

                if replace_value.strip():
                    # replace the search value with the replace value for the selected occurrence
                    rdi = rdi[:start_index] + replace_value + rdi[start_index + len(search_value):]

                    # update the line in the ACH file
                    lines[i] = line[:rdi_index] + rdi + line[rdi_index + 8:]

        # update the batch control total receiving DFI identification sum for the current batch
        if current_batch_number in batch_total_rdi:
            batch_total_rdi[current_batch_number] += int(rdi)

    elif record_type == '8':
        # update the file control total receiving DFI identification sum
        batch_rdi = batch_total_rdi.get(current_batch_number, 0)
        file_total_rdi += batch_rdi

        if first_file_control:
            # update the first file control record with the updated total receiving DFI identification sum
            lines[i] = line[:file_total_rdi_index] + '{:010d}'.format(file_total_rdi) + line[file_total_rdi_index + 10:]
            first_file_control = False

        # update the batch control total receiving DFI identification sum in the ACH file
        if current_batch_number in batch_total_rdi:
            # update the batch control record with the updated total receiving DFI identification sum for the current batch
            updated_line = line[:batch_total_rdi_index] + '{:010d}'.format(batch_total_rdi[current_batch_number]) + line[batch_total_rdi_index + 10:]
            lines[i] = updated_line


# update the file control total receiving DFI identification sum in the ACH file
for i, line in enumerate(lines):
    record_type = line[0]
    if record_type == '9':
        # update the file control record with the updated total receiving DFI identification sum
        updated_line = line[:file_total_rdi_index] + '{:010d}'.format(file_total_rdi) + line[file_total_rdi_index + 10:]
        lines[i] = updated_line
        break  # Stop updating after the first file control record

# write the updated lines to the ACH file
with open('CorrectNacha.ach', 'w') as f:
    f.writelines(lines)

# print the batch control total receiving DFI identification sum for each batch
for batch_number, total_rdi in batch_total_rdi.items():
    print("Batch Control Total Receiving DFI Identification Sum for Batch {}: {}".format(batch_number, total_rdi))

# print the file control total receiving DFI identification sum
print("File Control Total Receiving DFI Identification Sum: {}".format(file_total_rdi))


In [ ]:
# ask the user for the value to search and replace
search_value = input("Enter the value to search: ")

# define index for Amount field in Detail Record
amount_index = 29
batch_total_debit_index = 20
batch_total_credit_index = 32

# read the contents of the ACH file
with open('CorrectNacha.ach', 'r') as f:
    lines = f.readlines()

# initialize batch total debit and credit sums as dictionaries
batch_total_debit = {}
batch_total_credit = {}

# initialize file control total debit and credit sums
file_total_debit = 0.0
file_total_credit = 0.0

# initialize flag to track the first file control record
first_file_control = True

# loop through the lines of the ACH file and prompt for replace value if found in the Amount field of the Detail Record
for i, line in enumerate(lines):
    record_type = line[0]
    if record_type == '5':
        # extract the batch number from the Batch Header Record
        current_batch_number = line[87:94].strip()
        # initialize the batch total debit and credit sums for the current batch
        batch_total_debit[current_batch_number] = 0.0
        batch_total_credit[current_batch_number] = 0.0

    elif record_type == '6':
        # check if the search value is present in the Amount field
        amount = line[amount_index:amount_index + 10]

        if search_value in amount:
            # print the line where the search value is found
            print("Found search value in line: {}".format(line.strip()))

            # print the batch information
            entry_detail_sequence_number = line[87:94].strip()
            print("Search value is present in Batch {} and Sequence No {}.".format(current_batch_number,
                                                                                     entry_detail_sequence_number))

            # find the positions of all occurrences of the search value within the Amount field
            occurrence_positions = [index for index in range(len(amount)) if
                                    amount[index:index + len(search_value)] == search_value]

            # iterate through the occurrence positions and ask the user for the value to replace with
            for occurrence_index, start_index in enumerate(occurrence_positions):
                print("Occurrence {}: Position {}".format(occurrence_index + 1, start_index))
                replace_value = input("Enter the value to replace (leave blank to skip): ")

                if replace_value.strip():
                    # replace the search value with the replace value for the selected occurrence
                    amount = amount[:start_index] + replace_value + amount[start_index + len(search_value):]

                    # update the line in the ACH file
                    lines[i] = line[:amount_index] + amount + line[amount_index + 10:]

        # update the batch total debit and credit sum for the current batch
        if current_batch_number in batch_total_debit and current_batch_number in batch_total_credit:
            if line[1:3] == '27':
                batch_total_debit[current_batch_number] += float(amount) / 100
            elif line[1:3] == '22':
                batch_total_credit[current_batch_number] += float(amount) / 100

    elif record_type == '8':
        # update the file control total debit and credit sums
        if line[1:3] == '27':
            file_total_debit += float(line[20:32]) / 100
        elif line[1:3] == '22':
            file_total_credit += float(line[32:44]) / 100

        if first_file_control:
            # update the first file control record with the updated total debit and credit sums
            lines[i] = line[:32] + '{:012.0f}'.format(file_total_debit * 100) + '{:012.0f}'.format(
                file_total_credit * 100) + line[56:]
            first_file_control = False

# calculate the sum of all batch control debit and credit amounts for the file control
# Calculate the file control totals by summing the batch control totals
file_total_debit = sum(batch_total_debit.values())
file_total_credit = sum(batch_total_credit.values())

# update the batch control totals in the ACH file
for i, line in enumerate(lines):
    record_type = line[0]
    if record_type == '8':
        current_batch_number = line[87:94].strip()
        if current_batch_number in batch_total_debit and current_batch_number in batch_total_credit:
            # update the batch control record with the updated total debit and credit sums for the current batch
            updated_line = line[:batch_total_debit_index] + '{:012.0f}'.format(
                batch_total_debit[current_batch_number] * 100) + line[
                                                                   batch_total_debit_index + 12:batch_total_credit_index] + '{:012.0f}'.format(
                batch_total_credit[current_batch_number] * 100) + line[batch_total_credit_index + 12:]
            lines[i] = updated_line

# update the batch control totals in the first file control record
for i, line in enumerate(lines):
    record_type = line[0]
    if record_type == '9':
        # update the first file control record with the updated total debit and credit sums
        updated_line = line[:31] + '{:012.0f}'.format(file_total_debit * 100) + '{:012.0f}'.format(
            file_total_credit * 100) + line[43:]
        lines[i] = updated_line
        break  # Stop updating after the first file control record

# write the updated lines to the ACH file
with open('UpdatedNacha.ach', 'w') as f:
    f.writelines(lines)

# print the batch control totals for each batch
for batch_number in batch_total_debit:
    print("Batch Control Totals for Batch {}:".format(batch_number))
    print("Total Debit Amount: ${:.2f}".format(batch_total_debit[batch_number]))
    print("Total Credit Amount: ${:.2f}".format(batch_total_credit[batch_number]))

# print the file control totals
print("File Control Totals:")
print("Total Debit Amount: ${:.2f}".format(file_total_debit))
print("Total Credit Amount: ${:.2f}".format(file_total_credit))


In [ ]:
import pandas as pd
import re
import xlsxwriter
from tabulate import tabulate
from decimal import Decimal

# Open the ACH file and read the lines
with open('CorrectNacha.ach', 'r') as f:
    lines = f.readlines()

# Create a DataFrame for each record type
header_df = pd.DataFrame([{
    'Record Type': lines[0][0:1],
    'Priority Code': lines[0][1:3],
    'Immediate Destination': lines[0][3:13],
    'Immediate Origin': lines[0][13:23],
    'File Creation Date': f"{lines[0][23:25]}/{lines[0][25:27]}/{lines[0][27:29]}",
    'File Creation Time': f"{lines[0][29:31]}:{lines[0][31:33]}",
    'File ID Modifier': lines[0][33:34],
    'Record Size': lines[0][34:37],
    'Blocking Factor': lines[0][37:39],
    'Format Code': lines[0][39:40],
    'Immediate Destination Name': lines[0][40:63],
    'Immediate Origin Name': lines[0][63:86],
    'Reference Code': lines[0][86:94]
}], index=[0])


# Create a DataFrame to store the batch header record entries
batch_header_df = pd.DataFrame(columns=['Record Type', 'Service Class Code', 'Company Name',
                                        'Company Identification', 'Standard Entry Class Code',
                                        'Company Entry Description', 'Company Descriptive Date',
                                        'Effective Entry Date', 'Settlement Date', 'Originator Status Code',
                                        'Originating DFI Identification', 'Batch Number'])

# Iterate over the lines in the file and populate the DataFrames

from datetime import datetime, timedelta
import datetime

for i in range(0, len(lines), 1):
    if lines[i][0:1] == '5':
        # Map of Standard Entry Class Codes to their descriptions
        sec_code_map = {
            'PPD': 'Prearranged Payment and Deposit Entry',
            'CCD': 'Corporate Credit or Debit',
            'CTX': 'Corporate Trade Exchange',
            'WEB': 'Internet-initiated Entry',
            'TEL': 'Telephone-initiated Entry',
            'RCK': 'Re-presented Check Entry',
            'POP': 'Point of Purchase Entry',
            'ARC': 'Accounts Receivable Entry',
            'BOC': 'Back Office Conversion Entry',
            'MTE': 'Machine Transfer Entry',
            'POS': 'Point of Sale Entry',
            'SHR': 'Shared Network Entry',
            'ADV': 'Automated Accounting Advice'
        }
        
        sec_code = lines[i][50:53]
        sec_desc = sec_code_map.get(sec_code, 'Unknown')
        
        batch_header_df = batch_header_df.append({
            'Record Type': lines[i][0:1],
            'Service Class Code': lines[i][1:4],
            'Company Name': lines[i][4:20],
            'Company Discretionary Data': lines[i][20:40],
            'Company Identification': lines[i][40:50],
            'Standard Entry Class Code': sec_code + ':' + sec_desc,
            'Company Entry Description': lines[i][53:63],
            'Company Descriptive Date': '{}/{}/{}'.format(lines[i][63:65], lines[i][65:67], lines[i][67:69]),
            'Effective Entry Date': '{}/{}/{}'.format(lines[i][69:71], lines[i][71:73], lines[i][73:75]),
            'Settlement Date': lines[i][75:78],
            'Originator Status Code': lines[i][78:79],
            'Originating DFI Identification': lines[i][79:87],
            'Batch Number': lines[i][87:94]
        }, ignore_index=True)



# define a dictionary to store the transaction code descriptions
transaction_code_descriptions = {'21': 'ACH Credit',
                                 '22': 'Deposit destined for a Checking Account',
                                 '23': 'ACH Credit Pre-Note',
                                 '24': 'ACH Debit Pre-Note',
                                 '25': 'ACH Credit Reversal',
                                 '26': 'ACH Debit Reversal',
                                 '27': 'Debit destined for a Checking Account',
                                 '28': 'ACH Debit Reversal - NSF',
                                 '29': 'ACH Credit Correction',
                                 '31': 'ACH Savings Deposit',
                                 '32': 'Deposit destined for a Savings Account',
                                 '33': 'ACH Savings Payment',
                                 '34': 'ACH Savings Receipt',
                                 '35': 'ACH General Ledger Credit',
                                 '36': 'ACH General Ledger Debit',
                                 '37': 'ACH Loan Credit',
                                 '38': 'ACH Loan Debit'}

# loop through each line in the file and extract the relevant fields
# create an empty dictionary to hold the batch dataframes
batch_dataframes = {}

# loop through each line in the file and extract the relevant fields
batch_number = None
for i in range(1, len(lines) - 1, 1):
    if lines[i][0:1] == '5':
        # create a new empty dataframe for this batch
        batch_dataframes[lines[i][87:94]] = pd.DataFrame(columns=['Batch Number', 
                                   'Record Type', 
                                   'Transaction Code', 
                                   'Receiving DFI Identification', 
                                   'Check Digit', 'DFI Account Number', 
                                   'Amount', 'Individual ID Number', 'Individual Name', 
                                   'Addenda Indicator', 'Trace Number','Entry Detail Sequence Number'])
        batch_number = lines[i][87:94]
        
    elif lines[i][0:1] == '6':
        transaction_code = lines[i][1:3]
        transaction_description = transaction_code_descriptions.get(transaction_code, 'Invalid Transaction Code')
        entry_data = {
            'Batch Number': batch_number,
            'Record Type': lines[i][0:1],
            'Transaction Code': '{}: {}'.format(transaction_code, transaction_description),
            'Receiving DFI Identification': lines[i][3:11],
            'Check Digit': lines[i][11:12],
            'DFI Account Number': lines[i][12:29],
            'Amount': '${:.2f}'.format(float(lines[i][29:39])/100),
            'Individual ID Number': lines[i][39:54],
            'Individual Name': lines[i][54:76],
            'Discretionary Data': lines[i][76:78],
            'Addenda Indicator': lines[i][78:79],
            'Trace Number': lines[i][79:94],
            'Entry Detail Sequence Number':lines[i][87:94]
        }
        # add the entry data to the appropriate batch dataframe
        batch_dataframes[batch_number] = batch_dataframes[batch_number].append(entry_data, ignore_index=True)

# Initialize Batch Control Record table
# Batch Control Record
batch_control_df = pd.DataFrame(columns=['Record Type Code', 'Service Class Code', 'Entry/Addenda Count', 'Entry Hash','Receiving DFI Identification Sum',
                                          'Total Debit Amount', 'Total Credit Amount', 'Company Id',
                                          'Message Authentication Code', 'Reserved','Originating DFI Identification','Batch Number'])

batch_count = 0
receiving_dfi_sum = 0

for line in lines:
    record_type = line[0]
    if record_type == '5':
        batch_count += 1
    elif record_type == '6':
        receiving_dfi_sum += int(line[3:11])

    elif record_type == '8':
        service_class_code = line[1:4]
        if service_class_code == '200':
            service_class_code += ':ACH Entries Mixed Debits and Credits'
        elif service_class_code == '220':
            service_class_code += ':ACH Credits Only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        elif service_class_code == '280':
            service_class_code += ':ACH debits originated by the FI'
        elif service_class_code == '500':
            service_class_code += ':ACH debits originated by the FI, mixed addenda'
        elif service_class_code == '505':
            service_class_code += ':ACH debits originated by the FI, addenda only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        
        batch_control_df = batch_control_df.append({
            'Record Type Code': '8',
            'Service Class Code': service_class_code,
            'Entry/Addenda Count': line[4:10],
            'Entry Hash': line[10:20],
            'Receiving DFI Identification Sum': '{:.0f}'.format(receiving_dfi_sum),
            'Total Debit Amount':'${:.2f}'.format(float( line[20:32])/100),
            'Total Credit Amount':'${:.2f}'.format(float( line[32:44])/100),
            'Company Id': line[44:54],
            'Message Authentication Code': line[54:73],
            'Reserved': line[73:79],
            'Originating DFI Identification':line[79:87],
            'Batch Number':line[87:94]

        }, ignore_index=True)

        receiving_dfi_sum = 0

        if batch_count == 1:
            batch_control_df.iloc[-1, 0] = '8'
            batch_count = 0

# Remove the last row if it is empty
if batch_control_df.iloc[-1].isnull().all():
    batch_control_df = batch_control_df.iloc[:-1]

# Convert the receiving DFI sum to an integer
total_receiving_dfi_sum = int(batch_control_df['Receiving DFI Identification Sum'].str.replace('$', '').str.replace(',', '').astype(float).sum())

for line in lines:
    if line.startswith('90'):
        # Parse the line to extract the required fields
        record_type = line[0:1]
        batch_count = line[1:7]
        block_count = line[7:13]
        entry_addenda_count = line[13:21]
        entry_hash = line[21:31]
        total_receiving_dfi_sum = '{:010d}'.format(total_receiving_dfi_sum)
        total_debit_entry_dollar_amount = '${:.2f}'.format(float(line[31:43].strip('.').replace(' ', '0'))/100)
        total_credit_entry_dollar_amount = '${:.2f}'.format(float(line[43:55].strip('.').replace(' ', '0'))/100)

        reserved = line[55:94]
        
        # Create a DataFrame with the parsed fields
        file_control_df = pd.DataFrame([{
            'Record Type': record_type,
            'Batch Count': batch_count,
            'Block Count': block_count,
            'Entry/Addenda Count': entry_addenda_count,
            'Entry Hash': entry_hash,
            'Total Receiving DFI Identification Sum': total_receiving_dfi_sum,
            'Total Debit Entry Dollar Amount': total_debit_entry_dollar_amount,
            'Total Credit Entry Dollar Amount': total_credit_entry_dollar_amount,
            'Reserved': reserved
        }])
        
        # Compare the Entry Hash with Total Receiving DFI Identification Sum to validate the ACH file
        if entry_hash == total_receiving_dfi_sum:
            validity = "Valid"
        else:
            validity = "Not Valid"

        # Create the table with the output
        table = [["Entry Hash", entry_hash],
                 ["Total Receiving DFI Identification Sum", total_receiving_dfi_sum],
                 ["Validity", validity]]


from tabulate import tabulate

# calculate total debit amount and total credit amount from batch control
total_debit_amount = 0
total_credit_amount = 0
batchwise_total_debit_amount = {}
batchwise_total_credit_amount = {}
for index, row in batch_control_df.iterrows():
    debit_amount = float(row['Total Debit Amount'].replace('$', '').replace(',', ''))
    credit_amount = float(row['Total Credit Amount'].replace('$', '').replace(',', ''))
    total_debit_amount += debit_amount
    total_credit_amount += credit_amount
    batch_number = row['Batch Number']
    if batch_number not in batchwise_total_debit_amount:
        batchwise_total_debit_amount[batch_number] = 0
    if batch_number not in batchwise_total_credit_amount:
        batchwise_total_credit_amount[batch_number] = 0
    batchwise_total_debit_amount[batch_number] += debit_amount
    batchwise_total_credit_amount[batch_number] += credit_amount

# calculate total debit entry dollar amount and total credit entry dollar amount from file control
total_debit_entry_dollar_amount = float(file_control_df['Total Debit Entry Dollar Amount'].iloc[0].replace(',', '').replace('$', ''))
total_credit_entry_dollar_amount = float(file_control_df['Total Credit Entry Dollar Amount'].iloc[0].replace(',', '').replace('$', ''))

# check if total debit amount and total credit amount from batch control match with file control
if total_debit_amount == total_debit_entry_dollar_amount and total_credit_amount == total_credit_entry_dollar_amount:
    ach_status = "valid"
else:
    ach_status = "not valid"

# display batch wise totals

rows = []
for batch_number in batchwise_total_debit_amount.keys():
    rows.append([batch_number, '${:,.2f}'.format(batchwise_total_debit_amount[batch_number]), '${:,.2f}'.format(batchwise_total_credit_amount[batch_number])])

headers = ["Field", "Value"]
data = [
    ["Total Debit Amount from Batch Control", '${:,.2f}'.format(total_debit_amount)],
     ["Total Debit Entry Dollar Amount from File Control", '${:,.2f}'.format(total_debit_entry_dollar_amount)],
    ["Total Credit Amount from Batch Control", '${:,.2f}'.format(total_credit_amount)],
    ["Total Credit Entry Dollar Amount from File Control", '${:,.2f}'.format(total_credit_entry_dollar_amount)],
    ["ACH Status", ach_status]
]

# Define the expected record types
valid_header = '1'
valid_batch = '5'
valid_file_control = '9'

if lines[0].startswith(valid_header):
    # Check if file ends with a valid file control record
    if lines[-1].startswith(valid_file_control):
        # Check if file contains at least one batch record
        if any(line.startswith(valid_batch) for line in lines):
            file_validity = "Valid"
        else:
            file_validity = "Invalid"
    else:
        file_validity = "Invalid"
else:
    file_validity = "Invalid"


ach_table_headers = ['ACH File','Result']
ach_table_data = [[f.name ,file_validity]]
'''
'''
print(tabulate(ach_table_data, headers=ach_table_headers, tablefmt='fancy_grid'))
    
# Print the table
print(tabulate(table, headers=["Calculation", "Value"],tablefmt='fancy_grid'))

# print table using tabulate
print("\nBatch Wise Totals:")
headers = ["Batch Number", "Total Debit Amount", "Total Credit Amount"]
print(tabulate(rows, headers=headers,tablefmt='fancy_grid'))

print(tabulate(data, headers=headers,tablefmt='fancy_grid'))

if total_debit_amount == total_debit_entry_dollar_amount and total_credit_amount == total_credit_entry_dollar_amount:
    print("\nACH is valid")
else:
    reason = ""
    if total_debit_amount != total_debit_entry_dollar_amount:
        reason += "Total Debit Amount from Batch Control doesn't match with Total Debit Entry Dollar Amount from File Control.\n"
    if total_credit_amount != total_credit_entry_dollar_amount:
        reason += "Total Credit Amount from Batch Control doesn't match with Total Credit Entry Dollar Amount from File Control.\n"
    print("\nACH is not valid\nReason: ", reason)

print("\n Header Record: \n")
print(tabulate(header_df, headers='keys', tablefmt='fancy_grid'))

# Print the batch header DataFrame
print("\nBatch Header Record: \n")
print(tabulate(batch_header_df, headers='keys', tablefmt='fancy_grid'))

# Print each batch's entries in a separate table
for batch_number, df in batch_dataframes.items():
    print(f"\nEntries for Batch {batch_number}: \n")
    print(tabulate(df, headers='keys', tablefmt='fancy_grid'))

print("\nBatch Control Record: \n")
print(tabulate(batch_control_df, headers='keys', tablefmt='fancy_grid'))

print("\nFile Control Record: \n")
print(tabulate(file_control_df, headers='keys', tablefmt='fancy_grid'))

import pandas as pd

# Create a pandas Excel writer using the xlsxwriter engine
writer = pd.ExcelWriter('ACH_Exported_File.xlsx', engine='xlsxwriter')

# Define cell formats
header_format = writer.book.add_format({'bg_color': '#DDEBF7', 'bold': True, 'border': 1})
data_format = writer.book.add_format({'border': 1})

# Write header_df to Excel
header_df.to_excel(writer, sheet_name='Header', index=False, startrow=0, startcol=0, header=False)
worksheet = writer.sheets['Header']
for i, col in enumerate(header_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(header_df[col]):
        worksheet.write(j+1, i, value, data_format)

# Write batch_header_df to Excel
batch_header_df.to_excel(writer, sheet_name='Batch Header', index=False, startrow=0, startcol=0, header=False)
worksheet = writer.sheets['Batch Header']
for i, col in enumerate(batch_header_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(batch_header_df[col]):
        worksheet.write(j+1, i, value, data_format)

# Write the rest of the dataframes to Excel in separate tables
# write each batch to a separate sheet
for batch_number, df in batch_dataframes.items():
    df.to_excel(writer, sheet_name=f'Batch {batch_number}', index=False, header=True)

# format the worksheets
for worksheet_name in writer.sheets:
    worksheet = writer.sheets[worksheet_name]
    for i, col in enumerate(batch_dataframes[batch_number].columns):
        worksheet.write(0, i, col, header_format)
    for j, value in enumerate(df[col]):
        worksheet.write(j+1, i, value, data_format)

batch_control_df.to_excel(writer, sheet_name='Batch Control', index=False, startrow=0, startcol=0, header=True)
worksheet = writer.sheets['Batch Control']
for i, col in enumerate(batch_control_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(batch_control_df[col]):
        worksheet.write(j+1, i, value, data_format)

# Write the DataFrame to a sheet named 'File Control'
file_control_df.to_excel(writer, sheet_name='File Control', index=False, startrow=0, startcol=0, header=True)

# Apply the formats to the header row and data cells
worksheet = writer.sheets['File Control']
for i, col in enumerate(file_control_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(file_control_df[col]):
        worksheet.write(j+1, i, value, data_format)

# Save the Excel file and close the Pandas Excel writer
writer.save()
print('\n Data from ACH file Exported successfully and saved to ACH Exported File.xlsx \n')
